# To complete the project, you will need to merge the datasets, drop null values, and answer the questions below.

What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?

Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".
Store as a pandas DataFrame called adidas_vs_nike containing columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.

Do any differences exist between the word count of a product's description and its mean rating?

Split product description length into bins of 100-character intervals and calculate the average rating and total number of reviews.
Store the results as a pandas DataFrame called description_lengths containing columns: "description_length", "mean_rating", "total_reviews". Again, round numeric values to two decimal places.

In [13]:
import pandas as pd

brands = pd.read_csv("Data/brands.csv") 
finance = pd.read_csv("Data/finance.csv")
info = pd.read_csv("Data/info.csv")
reviews = pd.read_csv("Data/reviews.csv")

In [14]:
brands_finance = brands.merge(finance, on='product_id')
brands_finance.head()

,product_id,brand,listing_price,sale_price,discount,revenue
0,AH2430,NaN,NaN,NaN,NaN,NaN
1,G27341,Adidas,75.99,37.99,0.5,1641.17
2,CM0081,Adidas,9.99,5.99,0.4,398.93
3,B44832,Adidas,69.99,34.99,0.5,2204.37
4,D98205,Adidas,79.99,39.99,0.5,5182.70


In [15]:
info_brands_finance = pd.merge(info, brands_finance, on='product_id')
info_brands_finance.head()

,product_name,product_id,description,brand,listing_price,sale_price,discount,revenue
0,NaN,AH2430,NaN,NaN,NaN,NaN,NaN,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",Adidas,75.99,37.99,0.5,1641.17
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,Adidas,9.99,5.99,0.4,398.93
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",Adidas,69.99,34.99,0.5,2204.37
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,Adidas,79.99,39.99,0.5,5182.70


In [16]:
sports_clothing = pd.merge(info_brands_finance, reviews, on='product_id')
sports_clothing.head()

,product_name,product_id,description,brand,listing_price,sale_price,discount,revenue,rating,reviews
0,NaN,AH2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",Adidas,75.99,37.99,0.5,1641.17,3.3,24.0
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,Adidas,9.99,5.99,0.4,398.93,2.6,37.0
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",Adidas,69.99,34.99,0.5,2204.37,4.1,35.0
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,Adidas,79.99,39.99,0.5,5182.70,3.5,72.0


In [17]:
sports_clothing.count()

product_name     3120
product_id       3179
description      3117
brand            3120
listing_price    3120
sale_price       3120
discount         3120
revenue          3120
rating           3120
reviews          3120
dtype: int64

In [ ]:
# Prints null values for each column
sports_clothing.isna().sum()

product_name     59
product_id        0
description      62
brand            59
listing_price    59
sale_price       59
discount         59
revenue          59
rating           59
reviews          59
dtype: int64

In [ ]:
# Drop null values
sports_clothing = sports_clothing.dropna()
sports_clothing.count()

product_name     3117
product_id       3117
description      3117
brand            3117
listing_price    3117
sale_price       3117
discount         3117
revenue          3117
rating           3117
reviews          3117
dtype: int64

In [ ]:
# There are only two brands in the dataset
sports_clothing['brand'].unique()

array(['Adidas', 'Nike'], dtype=object)

In [21]:
sports_clothing['listing_price'].describe()

count    3117.000000
mean       69.786558
std        46.559051
min         0.000000
25%        45.990000
50%        59.990000
75%        89.990000
max       299.990000
Name: listing_price, dtype: float64

In [ ]:
# Create bin labels and use pd.qcut() to cut and categorize data according to
# quartiles (q=4). Can be done with pd.cut() but this is simpler.

bin_labels = ['Budget', 'Average', 'Expensive', 'Elite']
sports_clothing['price_label'] = pd.qcut(sports_clothing['listing_price'], q=4, labels=bin_labels)
sports_clothing[['price_label','listing_price']].head(5)

,price_label,listing_price
1,Expensive,75.99
2,Budget,9.99
3,Expensive,69.99
4,Expensive,79.99
5,Average,47.99


In [23]:
adidas_count = sports_clothing['brand']=='Adidas'
nike_count = sports_clothing['brand']=='Nike'
sports_clothing[adidas_count].shape[0]


2575

In [24]:
sports_clothing[nike_count].shape[0]

542

In [ ]:
# Long approach using pd.merge() 
# Used 'observed' parameter to show only present values after applying aggregation functions
# If false, lists all combinations including non-observable values

num_prod = sports_clothing.groupby(['brand','price_label'], as_index=False, observed=True).agg(num_products=('brand','count'))
mean_rev = sports_clothing.groupby(['brand','price_label'], as_index=False, observed=True)['revenue'].mean().round(2)
mean_rev.rename(columns={'revenue':'mean_revenue'}, inplace=True)

adidas_vs_nike = pd.merge(num_prod, mean_rev, how='left', on=['brand','price_label'])
adidas_vs_nike

,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33
5,Nike,Average,8,675.59
6,Nike,Expensive,47,500.56
7,Nike,Elite,130,1367.45


In [ ]:
# ALTERNATE APPROACH

# This is cleaner and I have overlooked the fact that
# you can add columns by chaining .agg() to the grouping

'''
adidas_vs_nike = (
    sports_clothing.groupby(
        ['brand','price_label'],
        as_index=False
    )
    .agg(
        num_products=('brand','count'),
        mean_revenue=('revenue','mean')
    )
)

adidas_vs_nike['mean_revenue'] = adidas_vs_nike['mean_revenue'].round(2)

adidas_vs_nike

'''